In [ ]:
import pathlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (project_loader as opl)

In [ ]:
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
import UnitsNet

In [ ]:
project_filenames = {
    'permian-a': 'Project-frankNstein_Permian_UTM13FT_0412_PjtDataFrame.ifrac',
    'gng': 'GnG_DemoProject_wDataFrames.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
def make_project_data_frames_by_id(project):
    return {df.ObjectId: df for df in project.DataFrames.Items}

project_net_data_frames_by_id = toolz.pipe(
    projects,
    toolz.valmap(make_project_data_frames_by_id),
)
project_net_data_frames_by_id

In [ ]:
def make_project_data_frames_by_name(project):
    return {df.Name: df for df in project.DataFrames.Items}

project_net_data_frames_by_name = toolz.pipe(
    projects,
    toolz.valmap(make_project_data_frames_by_name),
)
project_net_data_frames_by_name

In [ ]:
len(list(toolz.get('permian-a', project_net_data_frames_by_name)))

In [ ]:
len(list(toolz.get('gng', project_net_data_frames_by_name)))

In [ ]:
def extract_net_data_frame_identities(df):
    return {
        'object_id': str(df.ObjectId),
        'name': df.Name,
        'display_name': df.DisplayName,
    }

@toolz.curry
def build_net_data_frame_identities(ndfs):
    return toolz.map(extract_net_data_frame_identities, ndfs)
    
net_data_frame_identities = toolz.pipe(
    project_net_data_frames_by_name,
    toolz.valmap(lambda dfs_by_name: dfs_by_name.values()),
    toolz.valmap(build_net_data_frame_identities),
    toolz.valmap(list),
)
net_data_frame_identities

In [ ]:
def make_data_frame_identities_summary(item):
    project_name, ndf_id_sums = item
    summaries = toolz.pipe(
        ndf_id_sums,
        toolz.map(lambda ndf_id_sum: toolz.merge(ndf_id_sum, {'project': project_name})),
    )
    return project_name, summaries

data_frame_identities_summary = toolz.itemmap(make_data_frame_identities_summary, net_data_frame_identities)
data_frame_identities_summary

In [ ]:
pd.DataFrame(data=data_frame_identities_summary['permian-a'], 
             columns=['project', 'object_id', 'name', 'display_name'])

In [ ]:
pd.DataFrame(data=data_frame_identities_summary['gng'], 
             columns=['project', 'object_id', 'name', 'display_name'])

In [ ]:
permian_project_01 = toolz.get_in(['permian-a', 'Project Data Frame 01'], project_net_data_frames_by_name)

In [ ]:
permian_project_01.DisplayName if permian_project_01.DisplayName else 'Fooey'